In [11]:
import logging 


class Logger :
    def __init__(self) :
        self.levels = {
            20 : 'INFO', 
            30 : 'WARNING',
            40 : 'ERROR',
            50 : 'CRITICAL'
        }
        
    def log(self, level : int, filename : str, log_message : str) -> None :
        """
        Need not to provide a extension for filename.
        """
        print(filename)
        logging.basicConfig(
            level = level,
            format =  '%(asctime)s-15s %(user)-8s %(message)s',
            filename = "old.log")
        
        if level == 20 : 
            logging.info(log_message)

In [12]:
logger = Logger()
logger.log(20, "old", "hi")

old


In [43]:
logging.basicConfig(level = logging.WARNING, filename = "new")

In [46]:
logging.warning("name:hi")

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.9/logging/__init__.py", line 1079, in emit
    msg = self.format(record)
  File "/usr/lib/python3.9/logging/__init__.py", line 923, in format
    return fmt.format(record)
  File "/usr/lib/python3.9/logging/__init__.py", line 662, in format
    s = self.formatMessage(record)
  File "/usr/lib/python3.9/logging/__init__.py", line 631, in formatMessage
    return self._style.format(record)
  File "/usr/lib/python3.9/logging/__init__.py", line 430, in format
    return self._format(record)
  File "/usr/lib/python3.9/logging/__init__.py", line 426, in _format
    return self._fmt % record.__dict__
ValueError: unsupported format character '%' (0x25) at index 11
Call stack:
  File "/usr/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.9/dis

In [66]:
from datetime import datetime
import os 

class Logger(object) :
    """
    This class will be used to log the message into the logs folder 
    """
    def __init__(self) :
        pass
    
    def log(self,folder : str, filename : str, level : str, message : str) -> None :
        """
        Folder - Need not to create a new folder if the folder is not exist.
        filename : filename for the log.
        level : what is the level of the log(info, warning, error, critical).
        message : pass the log message.
        """
        
        os.chdir("/home/adminuser/internship_projects/phising_website/logs")
        if not os.path.isdir(folder) :
            os.makedirs(folder)    
        path = os.path.join(folder, filename)
        current_moment = datetime.now()
        date = current_moment.date()
        current_time = current_moment.strftime("%H:%M:%S")
        with open(path, 'a+') as file :
            file.write(f"{level} : {str(date)} : {str(current_time)} : {message} \n" )
            

In [61]:
logger = Logger()
logger.log("dummy", "dummy.log", "info", "dummy text for 1")

In [60]:
pwd

'/home/adminuser/internship_projects/phising_website/logs'

NameError: name 'window' is not defined

In [68]:
from Application_logger import logger


class Client_Rawdata_validation :
    def __init__(self, file : str, type_of_data : str) -> None :
        self.type_of_data = type_of_data
        if self.type_of_data == "training" :
            self.schema_file_path = "schema_for_training_data.json"
        else :
            self.schema_file_path = "schema_for_testing_data.json"
        self.logger = logger.Logger()
        self.file = file
        
    def values_for_validation(self) -> tuple :
        
        try : 
            self.logger.log("Rawdata_logs", f"schema_data.log", "info", f"Start reading the pre-defined schema file of {type_of_data}")
            with open(self.schema_file_path, "r") as file :
                schema_data = file.load()
            number_of_columns = schema_data.get('Number_of_columns')
            name = schema_data.get('Name_of_file')
            length_of_datestamp = schema_data.get('Length_of_datestamp_in_filename')
            length_of_timestamp = schema_data.get('Length_of_timestamp_in_filename')
            columns_name = schema_data.get('Column_names')
            message = f"Number of columns={number_of_columns}, Length of datestamp={length_of_datestamp}, Length of timestamp={length_of_timestamp}, Pattern={name}"
            self.logger.log("Rawdata_logs", "rawdata.log", "info", message)
        
        except ValueError as error :
            self.logger.log("Rawdata_logs", "rawdata.log", "error", "Value for the key is not presented inside the json file.")
            raise error
        
        except KeyError as error :
            self.logger.log("Rawdata_logs", "rawdata.log", "error", "Key is not found in the json file")
            raise error
        
        except Exception as error :
            self.logger.log("Rawdata_logs", "rawdata.log", "error", str(error))
            raise error
        
        else :
            self.logger.log("Rawdata_logs", "rawdata.log", "info", "Reading of the schema file is SuccessFull!!")
    
        finally :
            return number_of_columns, columns_name, length_of_datestamp, length_of_timestamp
        
    def regex_pattern_creation(self) -> str :
        pattern = "['phising']+['\_'']+[\d_]+[\d]+\.csv"
        return pattern
    
    def directory_creation(self) :
        """
        Directory Creation for the Good data and for the bad data, after segregating them after validation.
        Folder Names -> Good_Data/, Bad_Data/
        """
        try :
            if self.type_of_data == "training" :
                parent_folder = "Training_data_segregation"
            else :
                parent_folder = "Testing_data_segregation"
            
            path = os.path.join(parent_folder, "Good_Data")
            if not os.path.isdir(path) :
                os.makedirs(path)
                
            path = os.path.join(parent_folder, "Bad_Data")
            if not os.path.isdir(path) :
                os.makedirs(path)
        
        except OSError as error :
            self.logger.log("General_logs", "general.log", "error", f"Error while creating the file : {str(error)}")
            raise error
        
        else :
            self.logger.log("General_logs", "general.log", "info", f"Created the file successfully!!")
            